# Q1. Demonstrating forgeability of plain RSA

## Plain RSA

Plain RSA signatures are vulnerable to **existential forgery** because of the multiplicative homomorphic property of RSA: $Sign(m_1 \cdot m_2) = Sign(m_1) \cdot Sign(m_2)$.

To forge a signature for a specific message $b$ (which we call the "target") without asking the Oracle to sign $b$ directly, we can use a **Blinding Attack**:

1.  **Setup**: The adversary has the public key $(e, n)$ and the target message $b$.
2.  **Blinding**: The adversary picks a random value $r$ (the "blinding factor") such that $\gcd(r, n) = 1$.
3.  **Request**: The adversary computes a new message $m' = b \cdot r^e \pmod n$. This $m'$ looks random and is distinct from $b$. The adversary asks the Oracle to sign $m'$.
4.  **Sign**: The Oracle returns the signature $s' = (m')^d \pmod n$.
    * Mathematically, $s' \equiv (b \cdot r^e)^d \equiv b^d \cdot (r^e)^d \equiv b^d \cdot r \pmod n$.
5.  **Unblinding**: The adversary removes the blinding factor by computing $s = s' \cdot r^{-1} \pmod n$.
    * Substituting $s'$, we get $s \equiv (b^d \cdot r) \cdot r^{-1} \equiv b^d \pmod n$.

The result $s$ is a valid signature for $b$, and the Oracle was never asked to sign $b$ directly.

In [1]:
import random

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = extended_gcd(b % a, a)
        return g, x - (b // a) * y, y

def modinv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError("Modular inverse does not exist")
    return x % m


def is_prime(n, k=40):
    if n == 2 or n == 3: return True
    if n % 2 == 0 or n < 2: return False

    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_prime(nbits):
    while True:
        p = random.getrandbits(nbits)
        p |= (1 << (nbits - 1)) | 1 
        if is_prime(p):
            return p

def generate_keypair(nbits=512):
    p = generate_prime(nbits)
    q = generate_prime(nbits)
    while p == q:
        q = generate_prime(nbits)
        
    n = p * q
    e = 65537
    phi = (p - 1) * (q - 1)
    
    if gcd(e, phi) != 1:
        return generate_keypair(nbits)
        
    d = modinv(e, phi)
    return ((e, n), (d, n))


class Oracle:
    def __init__(self, sk):
        self.d, self.n = sk
        self.history = set()

    def sign(self, m):
        if m in self.history:
            raise ValueError(f"Oracle: I've already signed {m}!")
        self.history.add(m)
        return pow(m, self.d, self.n)


def exploit():
    print("-" * 30)

    pk, sk = generate_keypair(512)
    e, n = pk
    
    b = random.randint(2, n - 1)
    
    oracle = Oracle(sk)
    print(f"Target Message b: {b}")
    print(f"Public Key (e, n): ({e}, {n})")

    print("-" * 30)

    while True:
        r = random.randint(2, n - 1)
        if gcd(r, n) == 1:
            break
    print(f"Chosen blinding factor r: {r}")

    m_prime = (b * pow(r, e, n)) % n
    print(f"Blinded message m' to request: {m_prime}")

    if m_prime == b:
        print("Error: Random r resulted in no blinding.")
        return

    s_prime = oracle.sign(m_prime)
    print(f"Oracle returned s': {s_prime}")

    # s = s' * r^-1 mod n
    r_inv = modinv(r, n)
    s_forged = (s_prime * r_inv) % n
    print(f"Calculated forged signature s: {s_forged}")

    print("-" * 30)

    is_valid = pow(s_forged, e, n) == b
    asked_for_b = b in oracle.history

    print(f"Forgery Valid: {is_valid}")
    print(f"Was 'b' requested from Oracle? {asked_for_b}")

    if is_valid and not asked_for_b:
        print("SUCCESS: Existential Forgery complete!")
    else:
        print("FAIL: Attack failed :(")

exploit()

------------------------------
Target Message b: 55999814609109874582252798680664779206469018442581580777965001151733403769713075952763791409421661771207229413788992856117735030206139191951328081239674150180206422736136439513790891523403359958728146265512727863235938418546940779225127811659153159029805583323681742435265087720747683210302442452381527645724
Public Key (e, n): (65537, 59168104380396311030410798604761687306048901249837664865628624013924297556653357064496985869762111784058152047874871771689576176121374973740340250146229464075016039169468193884940734840993085271023230134939333244966982077535599261937662213680270894969537896197074947027418959671977269472696789953012533461133)
------------------------------
Chosen blinding factor r: 29846474400784449444769692818283237571918017280663607356496666252578660007152874458738260225137123678554609600334750923451691263095960593957211941209782249117989086282754853576863209809292361051270118243136460635962301775184565567255733880713219681

## RSA with a twist

The "Twist" requires us to forge a signature for a **specific target message** $b$ chosen by the challenger, without asking the Oracle to sign $b$ directly[cite: 23, 26].

To do this, we rely on the **Blinding Attack** using the multiplicative homomorphic property of RSA:

1.  **Blinding**: We pick a random $r$ and compute a "blinded" message $m' = b \cdot r^e \pmod n$.
2.  **Signing**: We ask the Oracle to sign $m'$. The Oracle records $m'$ (which is not $b$) and returns $s' = (m')^d \pmod n$.
3.  **Unblinding**: We compute the signature for $b$ by removing the blinding factor: $s = s' \cdot r^{-1} \pmod n$.

This works because:
$$s' \equiv (b \cdot r^e)^d \equiv b^d \cdot r^{ed} \equiv b^d \cdot r \pmod n$$
$$s \equiv s' \cdot r^{-1} \equiv (b^d \cdot r) \cdot r^{-1} \equiv b^d \pmod n$$

In [2]:
def generate_keypair(nbits=512):
    """Generates a valid RSA keypair."""
    while True:
        p = random.getrandbits(nbits) | 1
        if is_prime(p): break
    while True:
        q = random.getrandbits(nbits) | 1
        if is_prime(q) and p != q: break
        
    n = p * q
    e = 65537
    phi = (p - 1) * (q - 1)
    
    if gcd(e, phi) != 1:
        return generate_keypair(nbits)
        
    d = modinv(e, phi)
    return ((e, n), (d, n))


class GameOracle:
    def __init__(self, sk):
        self.d, self.n = sk
        self.history = set()

    def sign_request(self, m):
        if m in self.history:
            print(f"Oracle: I already signed {m}!")
        self.history.add(m)
        return pow(m, self.d, self.n)


def exploit_twist():
    print("-" * 30)
    pk, sk = generate_keypair(512)
    e, n = pk
    b = random.randint(2, n - 1)
    
    oracle = GameOracle(sk)
    print(f"Public Key (n): {n}")
    print(f"Target 'Bad Value' (b): {b}")

    print("-" * 30)
    
    while True:
        r = random.randint(2, n - 1)
        if gcd(r, n) == 1:
            break
            
    m_prime = (b * pow(r, e, n)) % n
    print(f"Adversary requests signature for m' (blinded): {m_prime}")

    if m_prime == b:
        print("Error: Blinding failed (random chance). Retry.")
        return
    
    s_prime = oracle.sign_request(m_prime)
    print(f"Oracle returns s': {s_prime}")

    print("-" * 30)
    
    r_inv = modinv(r, n)
    s_forged = (s_prime * r_inv) % n
    print(f"Adversary computes signature s for b: {s_forged}")

    print("-" * 30)
    
    is_valid_sig = pow(s_forged, e, n) == b
    print(f"i) Is s a valid signature for b? {is_valid_sig}")
    
    b_was_requested = b in oracle.history
    print(f"ii) Was b requested from Oracle? {b_was_requested}")
    
    if is_valid_sig and not b_was_requested:
        print("SUCCESS: The adversary successfully forged the signature for b!")
    else:
        print("FAIL: The adversary failed.")

exploit_twist()

------------------------------


Public Key (n): 55594975480505689549826578113699535888652650247182727373119296030002194870254111059970910139832223326815078435638262635582336436020795275675785247272784980631265301067760979371173232347880723430007625288266827072320740328708701248674488078528322428318612303383552330731526999378046263346127645425094620654511
Target 'Bad Value' (b): 20487554538024685836252793094157628418322284747350622022665168573561607196069665675342748176294162539258474878186526945090474888804954021031321480304777663708304913610645191774310028541319684531859562429794132911368622590495219037160671549832680154004720142564499813531997917964041751278989879662861824474116
------------------------------
Adversary requests signature for m' (blinded): 160337729498239751994254218403568448312687870961376762133729924517168137377645300021865752333007800278741561889966559541256947985205501309404520317071877342582061379167152009213116609357687139392498641461027116645225619827165426231171379048588183534339762978672718

# Q2. Shamir Secret Sharing

### Question - P1:

In [ ]:
# Need to use SageMath!!
def shamir_share(x):
    p = 208351617316091241234326746312124448251235562226470491514186331217050270460481
    F = GF(p)

    R.<t> = PolynomialRing(F)

    secret = F(x)

    # Quadratic polynomial
    a = F.random_element()
    b = F.random_element()
    f = secret + a*t + b*t^2

    # Generate 4 shares
    shares = [(i, f(i)) for i in [1, 2, 3, 4]]

    return shares


shares = shamir_share(123456)
print("Shares: ", shares)

# 1 - Degree should be quadratic
# 2 - To f(0) = x, then, in the expressions written above 'f = secret + a*t + b*t^2', when t = 0, then f(0) = x. In our case, will the 'secret'.


Shares:  [(1, 203700221403271140815870594359390484974431435734157392650580418258394226224001), (2, 98552734077157371546819669924694216526697695050851293456843162076002045605697), (3, 101260772653841174661500719320160091159269902403022685447160893886923999189506), (4, 3472719817231308925586996233663660620912495564201077107347282474109816514947)]


### Question - P2:

In [ ]:
import random

# This function takes n points and returns the unique polynomial 
# of degree n−1 that passes through them, using Lagrange interpolation.
def interpolate(points, F):
    R.<t> = PolynomialRing(F)
    f = R(0)

    for i, (xi, yi) in enumerate(points):
        Li = R(1)
        for j, (xj, _) in enumerate(points):
            if i != j:
                Li *= (t - xj) / (xi - xj)
        f += yi * Li

    return f


secret = 1001
shares = shamir_share(secret)
print(shares)

# The minimum number of shares needed is 3.
subset = random.sample(shares, 3)
print("Selected shares:", subset)

f_prime = interpolate(subset, F)
print("Recovered polynomial f'(t):", f_prime)


# The secret is stored at x=0
recovered_secret = f_prime(0)

print("Recovered secret:", recovered_secret)
print("Correct recovery:", recovered_secret == secret)

[(1, 128517608681365752147397921742571503814141757091657534628072682642005787575082), (2, 205485026444278228023287222831197048289233175775444669082542736994508636538217), (3, 22550635972646186393341156953752185174038693824890911849223831840458276429925), (4, 204769289214743350960539963046610259222264997919407737470674960831005518631649)]
Selected shares: [(4, 204769289214743350960539963046610259222264997919407737470674960831005518631649), (3, 22550635972646186393341156953752185174038693824890911849223831840458276429925), (1, 128517608681365752147397921742571503814141757091657534628072682642005787575082)]
Recovered polynomial f'(t): 182576521856864603098572435985151468581710393022535291427385017072298801155008*t^2 + 154292704140592390283152232069544483483666926295592734714873996786757256879554*t + 1001
Recovered secret: 1001
Correct recovery: True


### Question - P3:

In [18]:
secret1 = 100
shares_s1 = shamir_share(secret1)
print("Shares for secret 100:", shares_s1)

secret2 = 550
shares_s2 = shamir_share(secret2)
print("Shares for secret 550:", shares_s2)

# Combine first 3 shares
z_shares = [(shares_s1[i][0], shares_s1[i][1] + shares_s2[i][1]) for i in range(3)]
print("Combined shares z1, z2, z3:", z_shares)

# Recover polynomial from combined shares
s1_plus_s2 = interpolate(z_shares, F)
print("Recovered polynomial s1 + s2:", s1_plus_s2)

# Recover the secret
recovered_secret = s1_plus_s2(0)
print("Recovered secret:", recovered_secret)
print("Expected secret (100 + 550):", secret1 + secret2)



if recovered_secret == secret1 + secret2:
    print("Conclusion:\n\tThe secret is the sum of the individual secrets.\n\tThis demonstrates the homomorphic property of Shamir Secret Sharing.")
else:
    print("Something went wrong!")

Shares for secret 100: [(1, 16488565082969002684164139058521676394833459790965845120498494701880016383995), (2, 71511822733216829229125960310003369460551283619281889350011782670527916241940), (3, 165069772950743479634885463754445079197153471484948132688539863905943699573935), (4, 88810798419457712667115903079722357353404461161494083621896407191077095919499)]
Shares for secret 550: [(1, 88737234733717696402675224530497217918895971828778393494266874015180940608097), (2, 83606848470948414617078878895009350026381649380367335087575544257828615388286), (3, 192960458527783395877537709405660844573692594881237316294112341944993294801598), (4, 94830272040157715398223438202805058357683878447354085504604642574437927071)]
Combined shares z1, z2, z3: [(1, 105225799816686699086839363589018894313729431619744238614765368717060956992092), (2, 155118671204165243846204839205012719486932932999649224437587326928356531630226), (3, 149678614162435634278096426847981475519610504139714957468465874633886723915052